In [27]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ta.momentum import RSIIndicator
from ta.trend import MACD
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
import os

In [28]:
stocks = ["RELIANCE.NS", "HDFCBANK.NS", "INFY.NS"]
stock_data = {}

In [32]:
for stock in stocks:
    if os.path.exists(f"{stock}.csv"):
        stock_data[stock] = pd.read_csv(f"{stock}.csv", index_col=0, parse_dates=True)
    else:
        ticker = yf.Ticker(stock)
        stock_data[stock] = ticker.history(period="max")
        stock_data[stock].to_csv(f"{stock}.csv")

In [33]:
for stock in stocks:
    print(f"\nBasic Statistics for {stock}:")
    print(stock_data[stock][["Open", "High", "Low", "Close", "Volume"]].describe())


Basic Statistics for RELIANCE.NS:
              Open         High          Low        Close        Volume
count  7428.000000  7428.000000  7428.000000  7428.000000  7.428000e+03
mean    330.606448   334.200081   326.786256   330.358959  5.872732e+07
std     418.435364   422.302549   414.404913   418.221028  1.014932e+08
min       3.447046     3.462817     3.425643     3.462817  0.000000e+00
25%      19.602716    19.868367    19.320005    19.591828  1.347409e+07
50%     183.592594   185.615889   180.853095   183.490685  2.377707e+07
75%     410.233114   413.387463   406.147393   408.630997  6.121514e+07
max    1599.022925  1603.358288  1580.137072  1595.484985  1.448889e+09

Basic Statistics for HDFCBANK.NS:
              Open         High          Low        Close        Volume
count  7431.000000  7431.000000  7431.000000  7431.000000  7.431000e+03
mean    460.769056   465.090833   456.226109   460.741536  6.049395e+06
std     560.059051   564.701241   555.357269   560.046262  7.77269

In [37]:
fig = make_subplots(
    rows=3,
    cols=1,
    subplot_titles=[f"{stock} Closing Price and Volume" for stock in stocks],
    vertical_spacing=0.1,
    specs=[[{'secondary_y': True}] for _ in range(3)]  # Enable secondary_y for each subplot
)

for i, stock in enumerate(stocks, 1):
    df = stock_data[stock]
    # Add candlestick chart for stock price
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df["Open"],
            high=df["High"],
            low=df["Low"],
            close=df["Close"],
            name=stock,
            increasing_line_color="#00CC96",
            decreasing_line_color="#EF553B",
        ),
        row=i,
        col=1,
        secondary_y=False,  # Primary y-axis for price
    )
    # Add volume bar chart
    fig.add_trace(
        go.Bar(
            x=df.index,
            y=df["Volume"],
            name=f"{stock} Volume",
            marker_color="#636EFA",
            opacity=0.3,
        ),
        row=i,
        col=1,
        secondary_y=True,  # Secondary y-axis for volume
    )

# Update layout for a professional look
fig.update_layout(
    title="Nifty50 Stocks: Price and Volume (Last 5 Years)",
    height=1200,
    showlegend=False,
    template="plotly_dark",
    xaxis_rangeslider_visible=False,
)
fig.update_yaxes(title_text="Price (INR)", secondary_y=False)
fig.update_yaxes(title_text="Volume", secondary_y=True)

# Save and display the plot
fig.write_html("nifty50_eda.html")
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed